In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
from cloudpickle import loads

def handler(context, event):
    context.logger.info("Loading Model")
    print(str(context.inputs['model']))
    print(context.inputs['model'].get())
    print(type(context.inputs['model'].get()))
    
    return
    model = torch.load(str(context.inputs['model']))
#     model = loads(context.inputs['model'].get())
    model.eval()
    
    context.logger.info("Loading Data Loader")
    test_data_loader = loads(context.inputs['test_data_loader'].get())
    
    context.logger.info("Inferencing Model")
    num_correct = 0 
    num_examples = 0
    test_loss = 0.0
    device = str(context.inputs['device'])
    with torch.no_grad():    
        for batch in test_data_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            output = model(inputs)
            targets = targets.to(device)
            
            loss = nn.CrossEntropyLoss()(output,targets) 
            test_loss += loss.data.item() * inputs.size(0)
                        
            correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1], targets).view(-1)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        test_loss /= len(test_data_loader.dataset)        
                
    context.logger.info("Logging Results")
    context.logger.info('Test Loss: {:.4f}, Test Accuracy = {:.4f}'.format(test_loss, num_correct / num_examples))
    results = {"test_loss" : test_loss,
               "test_accuracy" : num_correct / num_examples}
    context.log_results(results)